**IMPORTING RELEVANT LIBRARIES**

In [2]:
%pip install wordcloud imbalanced-learn
%pip install -U imbalanced-learn
%pip install -U scikit-learn
%pip install -U seaborn
%pip install -U matplotlib
%pip install nltk
%pip install -U pandas
%pip install -U numpy
%pip install -U matplotlib
%pip install -U wordcloud
%pip install -U tqdm
%pip install -U pyspellchecker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import re # Regular expressions
import nltk
from nltk.corpus import stopwords # Stop words removal
from nltk.tokenize import word_tokenize # Word tokenization
from nltk.stem import WordNetLemmatizer  # Word lemmatization
# Download NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from wordcloud import WordCloud

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF vectorization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample, shuffle
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to C:\Users\Taqi
[nltk_data]     Haider\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Taqi
[nltk_data]     Haider\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Taqi
[nltk_data]     Haider\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Taqi
[nltk_data]     Haider\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Loading the dataset
data = pd.read_csv("amazon-updated.csv")


In [4]:
# Checking the shape
data.shape

(1965, 16)

In [5]:
#Checking the Cloumns
data.columns

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'img_link', 'product_link'],
      dtype='object')

In [6]:
# Previewing the dataset
data.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [ ]:
# # Checking the Scores/Ratings Distribution
# data.Score.value_counts()

# Tried But not work

In [10]:
# Checking the Ratings Distribution
# print(data.columns)  # Check available columns

# If the column is 'rating' (lowercase), use that:
data['rating'].value_counts()

rating
4.1    244
4.3    230
4.2    228
4.0    129
3.9    123
4.4    123
3.8     86
4.5     75
4       52
3.7     42
2.7     37
3.6     35
1.6     34
2.1     33
1.4     33
1.2     30
2.0     29
2.8     28
2.6     27
1.5     27
1.3     26
3.5     26
1.7     26
1.1     24
2.4     22
1.9     22
2.3     22
1.8     22
2.2     21
2.9     19
2.5     17
4.6     17
3.3     16
3.4     10
4.7      6
1.0      6
3.1      4
3.0      3
4.8      3
5.0      3
3.2      2
2        1
|        1
3        1
Name: count, dtype: int64

In [ ]:
# Checking for Missing Values
data.isnull().sum()

In [ ]:
# Dropping the Rows with Missing Values
data.dropna(inplace=True)

In [ ]:
# Checking the shape again
data.shape

In [ ]:
# Removing the unnecessary features and retaining the relevant features for Sentiment Analysis
data = data[['Score','Summary','Text']]

In [ ]:
# Previewing the dataset again
data.head()

In [ ]:
# Making the copy of the dataset
df = data.copy()

In [ ]:
# Concatenating the Summary and Text features
df['Review'] = df['Summary'] + " " + df['Text']
df = df.drop(['Summary','Text'],axis=1)
df.head()

In [ ]:
# Cleaning the Dataset

def clean_text(text):
    text = re.sub(r'<.*?>', '', text) # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
    text = text.lower() # Convert to lowercase
    return text

df['Review'] = df['Review'].apply(clean_text)
df.head()

In [ ]:
# Initializing the stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Defining a function to preprocess the text
def preprocess_text(review):
    # Tokenization
    tokens = word_tokenize(review)

    # Stopword Removal
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    # Lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

     # Join the tokens back into a string
    cleaned_review = ' '.join(lemmatized_tokens)

    return cleaned_review

# Applying the function to the Review column
df['cleaned_review'] = df['Review'].apply(preprocess_text)

df = df.drop(['Review'], axis=1)

In [ ]:
# Previewing the data
df.head()

In [ ]:
# Combining all cleaned reviews into a single string
text = ' '.join(df['cleaned_review'].astype(str).tolist())

# Creating a WordCloud object
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stop_words, # Use the stop words you defined earlier
                min_font_size = 10).generate(text)

# Plotting the WordCloud image
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
# Defining sentiment labels based on the rating
def get_sentiment(rating):
    if rating < 3.0:
        return 'negative'
    elif rating >= 4.0:
        return 'positive'
    else:
        return 'neutral'

# Applying the function to create a new sentiment column
df['sentiment'] = df['Score'].apply(get_sentiment)

# Dropping the rating column as it has served its purpose
df = df.drop(['Score'], axis=1)

df.head()

In [ ]:
# Checking for class distribution
print(df['sentiment'].value_counts())

# Visualize the distribution
sns.countplot(x='sentiment', data=df)
plt.title('Sentiment Class Distribution')
plt.show()

In [ ]:
# Encode the sentiment labels using LabelEncoder
encoder = LabelEncoder()
df['sentiment'] = encoder.fit_transform(df['sentiment'])
df.head()

In [ ]:
# Separate the classes
positive_reviews = df[df['sentiment'] == 2]
neutral_reviews = df[df['sentiment'] == 1]
negative_reviews = df[df['sentiment'] == 0]

# Downsample positive reviews
positive_reviews_downsampled = resample(positive_reviews,
                                        replace=False,     # sample without replacement
                                        n_samples=85000,   # match the negative sample size
                                        random_state=42)   # for reproducibility

# Combine downsampled positives with original neutral and negative
balanced_df = pd.concat([positive_reviews_downsampled, neutral_reviews, negative_reviews])

In [ ]:
balanced_df.shape

In [ ]:
print(balanced_df.value_counts('sentiment'))

# Visualize the distribution
sns.countplot(x='sentiment', data=balanced_df)
plt.title('Sentiment Class Distribution')
plt.show()

In [ ]:
# Shuffling the data
balanced_df = shuffle(balanced_df, random_state=42)

In [ ]:
# Splitting the dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(balanced_df['cleaned_review'], balanced_df['sentiment'], stratify=balanced_df['sentiment'], train_size=0.8 , test_size=0.2, random_state=42)

In [ ]:
#TF-IDF Vectorization
vect = TfidfVectorizer(max_features=7500, ngram_range=(1, 2), max_df=0.85, min_df=2)
X_train_vectorized = vect.fit_transform(X_train)  # Fit on training data
X_test_vectorized = vect.transform(X_test)        # Transform test data with the same vectorizer


In [ ]:
import pickle
# Save the vectorized datasets to a file using pickle
with open('vectorized_data.pkl', 'wb') as file:
    pickle.dump(vect, file)


In [ ]:
# Checking the Vocabulary Size
vocab_size = len(vect.vocabulary_)
print("Vocabulary size:", vocab_size)

In [ ]:
#SMOTE Upsampling on Training Data
smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_vectorized, y_train)

# LOGISTIC REGRESSION

In [ ]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(X_train_balanced, y_train_balanced)

best_params = grid_search.best_params_

best_model = grid_search.best_estimator_

In [ ]:
print("Best Hyperparameters:", best_params)
print(best_model)

In [ ]:
# Initialize the logistic regression model with L2 regularization (default for 'penalty' is 'l2')
log_reg_model = LogisticRegression(penalty='l2', C=10, solver='lbfgs', max_iter=1000, random_state=42)

# Train the model on the vectorized training data
log_reg_model.fit(X_train_balanced, y_train_balanced)


In [ ]:
# Predict on the vectorized test set
y_pred = log_reg_model.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
with open('log_reg_model.pkl', 'wb') as file:
    pickle.dump(log_reg_model, file)

# SVM(SUPPORT VECTOR MACHINE

In [ ]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_balanced, y_train_balanced)

In [ ]:
y_pred = svm_model.predict(X_test_vectorized)

In [ ]:
svm_accuracy = accuracy_score(y_test, y_pred)
print("SVM Model Accuracy:", svm_accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
with open('SVM.pkl', 'wb') as file:
    pickle.dump(svm_model, file)

# DATAFRAME VISUALIZATION (ACCURACY AND LOSS)

In [ ]:
data = {
    'Model': ['Logistic Regression', 'SVM'],
    'Accuracy': [0.7899, 0.7884],
    'Loss':[1-0.7899, 1-0.7884]
}
df = pd.DataFrame(data)

In [ ]:
# Create a grouped bar chart
x = np.arange(len(df['Model']))  # Position of groups on the x-axis
width = 0.35  # Width of the bars

# Initialize the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot bars for Accuracy and Loss
accuracy_bars = ax.bar(x - width/2, df['Accuracy'], width, label='Accuracy', color='skyblue')
loss_bars = ax.bar(x + width/2, df['Loss'], width, label='Loss', color='orange')

# Add labels and title
ax.set_xlabel('Model')
ax.set_ylabel('Value')
ax.set_title('Model Accuracies and Losses')
ax.set_xticks(x)
ax.set_xticklabels(df['Model'])
ax.legend()

# Add value annotations on top of bars
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',  # Text to display
                    xy=(bar.get_x() + bar.get_width() / 2, height),  # Position
                    xytext=(0, 3),  # Offset
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(accuracy_bars)
add_labels(loss_bars)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:

metrics_dict = {
    'Model': ['Logistic Regression', 'SVM'],
    'Accuracy': ['79%', '79%'],
    'Precision': ['80%', '80%'],
    'Recall': ['79%', '79%'],
    'F1-Score': ['79%', '79%'],
    'TrainingTime (mins)': [25, 180]
}

metrics_df = pd.DataFrame(metrics_dict)
metrics_df
